# Identifying and Tracking Solar Magnetic Flux Elements with Deep Learning

### Import Library's

In [27]:
import keras as keras
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from astropy.io import fits

## Model

### **Input:** This will have one neuron with a 720x720 image input

In [28]:
inputs = keras.Input(shape=(720, 720, 1))

### **Encoder 1:** Will have 32 Neurons kernel will be 3x3 with batch normalization and dropout at the end of the encoder with a connection to the 4th decoder

In [29]:
e1 = keras.layers.Conv2D(32, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
e1_bn = keras.layers.BatchNormalization()(e1)
e1 = keras.layers.Conv2D(32, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(e1_bn)
e1_bn = keras.layers.BatchNormalization()(e1)
e1_pool = keras.layers.MaxPooling2D(pool_size=(2,2))(e1_bn)
e1_dropout = keras.layers.Dropout(0.5)(e1_pool)

### **Encoder 2:** Will have 64 Neurons kernel will be 3x3 with batch normalization and dropout at the end of the encoder with a connection to the 3rd decoder

In [30]:
e2 = keras.layers.Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(e1_dropout)
e2_bn = keras.layers.BatchNormalization()(e2)
e2 = keras.layers.Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(e2_bn)
e2_bn = keras.layers.BatchNormalization()(e2)
e2_pool = keras.layers.MaxPooling2D(pool_size=(2,2))(e2_bn)
e2_dropout = keras.layers.Dropout(0.5)(e2_pool)

### **Encoder 3:** Will have 128 Neurons kernel will be 3x3 with batch normalization and dropout at the end of the encoder with a connection to the 2nd decoder

In [31]:
e3 = keras.layers.Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(e2_dropout)
e3_bn = keras.layers.BatchNormalization()(e3)
e3 = keras.layers.Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(e3_bn)
e3_bn = keras.layers.BatchNormalization()(e3)
e3_pool = keras.layers.MaxPooling2D(pool_size=(2,2))(e3_bn)
e3_dropout = keras.layers.Dropout(0.5)(e3_pool)

### **Encoder 4:** Will have 256 Neurons kernel will be 3x3 with batch normalization and dropout at the end of the encoder with a connection to the 1st decoder

In [32]:
e4 = keras.layers.Conv2D(256, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(e3_dropout)
e4_bn = keras.layers.BatchNormalization()(e4)
e4 = keras.layers.Conv2D(256, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(e4_bn)
e4_bn = keras.layers.BatchNormalization()(e4)
e4_pool = keras.layers.MaxPooling2D(pool_size=(2,2))(e4_bn)
e4_dropout = keras.layers.Dropout(0.5)(e4_pool)

### **Bottel Neck:** Will have 512 Neurons kernel will be 3x3 with batch normalization and dropout at the end of the encoder with a connection to the 1st decoder

In [34]:
bn = keras.layers.Conv2D(256, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(e4_dropout)
bn_bn = keras.layers.BatchNormalization()(bn)

### **Decoder 1:** Will have 256 Neurons kernel will be 2x2

In [36]:
d1 = keras.layers.Conv2D(256, (2,2), activation='relu', padding='same', kernel_initializer='he_normal')(keras.layers.UpSampling2D(size=(2,2))(bn_bn))
d1_merge = keras.layers.concatenate([e4, d1], axis=3)
d1_drop = keras.layers.Dropout(0.5)(d1_merge)
d1 = keras.layers.Conv2D(256, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(d1_drop)
d1_bn = keras.layers.BatchNormalization()(d1)
d1 = keras.layers.Conv2D(256, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(d1_bn)
d1_bn = keras.layers.BatchNormalization()(d1)

### **Decoder 2:** Will have 128 Neurons kernel will be 2x2

In [39]:
d2 = keras.layers.Conv2D(128, (2,2), activation='relu', padding='same', kernel_initializer='he_normal')(keras.layers.UpSampling2D(size=(2,2))(d1_bn))
d2_merge = keras.layers.concatenate([e3, d2], axis=3)
d2_drop = keras.layers.Dropout(0.5)(d2_merge)
d2 = keras.layers.Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(d2_drop)
d2_bn = keras.layers.BatchNormalization()(d2)
d2 = keras.layers.Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(d2_bn)
d2_bn = keras.layers.BatchNormalization()(d2)

### **Decoder 3:** Will have 128 Neurons kernel will be 2x2

In [41]:
d3 = keras.layers.Conv2D(64, (2,2), activation='relu', padding='same', kernel_initializer='he_normal')(keras.layers.UpSampling2D(size=(2,2))(d2_bn))
d3_merge = keras.layers.concatenate([e2, d3], axis=3)
d3_drop = keras.layers.Dropout(0.5)(d3_merge)
d3 = keras.layers.Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(d3_drop)
d3_bn = keras.layers.BatchNormalization()(d3)
d3 = keras.layers.Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(d3_bn)
d3_bn = keras.layers.BatchNormalization()(d3)

### **Decoder 4:** Will have 128 Neurons kernel will be 2x2

In [42]:
d4 = keras.layers.Conv2D(64, (2,2), activation='relu', padding='same', kernel_initializer='he_normal')(keras.layers.UpSampling2D(size=(2,2))(d3_bn))
d4_merge = keras.layers.concatenate([e1, d4], axis=3)
d4_drop = keras.layers.Dropout(0.5)(d4_merge)
d4 = keras.layers.Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(d4_drop)
d4_bn = keras.layers.BatchNormalization()(d4)
d4 = keras.layers.Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(d4_bn)
d4_bn = keras.layers.BatchNormalization()(d4)

## Output

In [43]:
outputs = keras.layers.Conv2D(1, (1,1), activation='sigmoid')(d4_bn)

In [45]:
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [46]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 720, 720, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_11 (Conv2D)             (None, 720, 720, 32  320         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_9 (BatchNo  (None, 720, 720, 32  128        ['conv2d_11[0][0]']              
 rmalization)                   )                                                           